In [26]:
import AnalyticalLabware
import logging

logging.basicConfig(level=logging.DEBUG)

In [27]:
from chempiler import Chempiler
import ChemputerAPI

c = Chempiler(
    experiment_code="oxazole",
    output_dir="\logs",
    graph_file="mini_AH_analytics.json",
    device_modules=[ChemputerAPI, AnalyticalLabware],
    simulation=True,
)

INFO:hplc_logger:Send command No. 256: last_cmd_no = 0.
INFO:hplc_logger:Reply: 
256 ACK
256 
256 DONE

DEBUG:hplc_logger:Reset command counter
INFO:hplc_logger:HPLC Controller initialized.
INFO:main_logger.sim_logger:Received SimIKARCTDigital hotplate (port = 5000)
INFO:main_logger.sim_logger:Received Valve "valve_1" Address: 192.168.1.41
INFO:main_logger.sim_logger:Received Valve "valve_2" Address: 192.168.1.42
INFO:main_logger.sim_logger:Received Valve "valve_3" Address: 192.168.1.43
DEBUG:main_logger.sim_logger:Valve "valve_1" - Waiting until ready...
DEBUG:main_logger.sim_logger:Valve "valve_2" - Waiting until ready...
DEBUG:main_logger.sim_logger:Valve "valve_3" - Waiting until ready...
INFO:main_logger.sim_logger:Received Pump "pump_1" Address: 192.168.1.12
INFO:main_logger.sim_logger:Received Pump "pump_2" Address: 192.168.1.14
INFO:main_logger.sim_logger:Received Pump "pump_3" Address: 192.168.1.15
DEBUG:main_logger.sim_logger:Pump "pump_1" - Waiting until ready...
DEBUG:main_

In [29]:
c["hotplate"].stop_stirrer()

INFO:main_logger.sim_logger:IKARCT - Stopping stirrer


In [28]:
c['idex'].is_connected

DEBUG:main_logger.serial_device_logger:Sending command S00
 sent.main_logger.serial_device_logger:Command  S00
DEBUG:main_logger.serial_device_logger:Reply requested, waiting for device...
DEBUG:main_logger.serial_device_logger:Received reply: 01


True

In [ ]:
from xdl import XDL
import ChemputerXDL

xdl_obj = XDL('van_leusen_oxazole.xdl')

In [ ]:
print(xdl_obj.human_readable())

In [ ]:
xdl_obj.prepare_for_execution('mini_AH.json')

In [ ]:
for base_step in xdl_obj.steps[1].base_steps:
    print(base_step, '\n', base_step.properties, '\n')

In [ ]:
from chempiler import Chempiler
import ChemputerAPI

c = Chempiler(
    experiment_code="oxazole",
    output_dir="\logs",
    graph_file="mini_AH.json",
    device_modules=[ChemputerAPI],
    simulation=False,
)

In [ ]:
xdl_obj.execute(c)

In [ ]:
c.move("argon", "waste_3", 5, speed=50)

In [ ]:
c.move("water", "waste_2", 5, speed=20)
c.move("water", "reactor", 5, speed=20)
c.move("argon", "reactor", 5, speed=20)

In [ ]:
c.move("reactor", "waste_2", 1, speed=20)

In [ ]:
c.move("reactor", "dilution", 1, speed=20)

In [ ]:
c.move("argon", "dilution", 1, speed=20)

In [ ]:
c.move("acetonitrile", "dilution", 9, speed=20)

In [ ]:
c.move("dilution", "waste_3", 8, speed=20)
c.move("acetonitrile", "dilution", 9, speed=20)

c.move("dilution", "waste_3", 8, speed=20)
c.move("acetonitrile", "dilution", 9, speed=20)

In [ ]:
c.move("dilution", "idex", 1, initial_pump_speed=20, end_pump_speed=0.5)

In [ ]:
c['idex'].sample(30)

In [ ]:
c['idex'].move_to_position(1)

In [ ]:
c.move("dilution", "waste_3", 10, speed=50)

In [ ]:
c.move("water", "dilution", 10, speed=50)

In [ ]:
def clean_dilution():
    c.move("flask_methanol", "dilution", 4, initial_pump_speed=10)
    c.move("dilution", "waste_3", 5, initial_pump_speed=10)
    c.move("acetonitrile", "dilution", 4, initial_pump_speed=10)
    c.move("dilution", "waste_3", 5, initial_pump_speed=10)
    c.move("water", "dilution", 4, initial_pump_speed=10)
    c.move("dilution", "waste_3", 5, initial_pump_speed=10)
    
def clean_idex():
    c.move("acetonitrile", "idex", 10, speed=10)
    # c.move("argon", "idex", 10, speed=20)

In [ ]:
clean_dilution()

In [ ]:
c.move("flask_methanol", "dilution", 4, initial_pump_speed=10, dest_port="0")

In [ ]:
def sample(dilution_vol=10):
    #make sure backbone is clean beforehand
    CleanBackbone(solvent="acetonitrile").execute(c)
    # dry with argon?
    
    #HPLC prep run

    
    c.breakpoint()
    
    # get sample
    c.move("reactor", "waste_2", 0.5, initial_pump_speed=10)
    c.move("reactor", "dilution", 0.1, initial_pump_speed=10, dest_port=0)
    c.move("argon", "dilution", 5, speed=50, dest_port=0)
    
    c.breakpoint()
    
    # dilute sample
    # potential chempiler bug regarding dest_port=0 when competing direct connection
    c.move("acetonitrile", "dilution", dilution_vol, initial_pump_speed=10, through_nodes="valve_2")
    c.wait(60)
    #TODO: Stirring
    
    c.breakpoint()
    
    # load sample loop
    c.move("dilution", "pump_3", 5, initial_pump_speed=10, src_port=1)
    c.move("pump_3", "idex", 2.5, speed=0.5)
    
    c.breakpoint()
    
    # trigger run
    c["hplc"].switch_method(name="AH_default")
    c["hplc"].preprun()
    while not c['hplc'].status()[0] == 'PRERUN':
        c.wait(1)       
    c["idex"].sample(10)
    c.wait(28*60) 
    
    
    c.move("pump_3", "waste_3", 2.5, speed=50)
    
    # async set back to standby when done
    #c["hplc"].standby()
            

    
    
def clean_hplc():
    
    #HPLC prep run
    c["hplc"].switch_method(name="AH_cleaning")
    c["hplc"].preprun()
    
    c.move("dilution", "waste_3", 10, speed=50, src_port=1)
    
    c.breakpoint()
    
#    for _ in range(3):
#        c.move("acetonitrile", "dilution", 10, speed=50, through_nodes="valve_2")
#        c.move("dilution", "waste_3", 15, speed=50, src_port=1)
        
    c.breakpoint()
        
    c.move("acetonitrile", "dilution", 5, speed=50, through_nodes="valve_2")
    # load sample loop
    c.move("dilution", "pump_3", 5, initial_pump_speed=10, src_port=1)
    c.move("pump_3", "idex", 2.5, speed=0.5)
    
    c.breakpoint()
    
    # trigger run
    while not c['hplc'].status()[0] == 'PRERUN':
        c.wait(1)
        
    c["idex"].sample(10)
    
    c.move("pump_3", "waste_3", 2.5, speed=50)    
    
    c.breakpoint()
    # async set back to standby when done
    c["hplc"].standby()
    c.move("dilution", "waste_3", 10, speed=50, src_port=1)

In [ ]:
sample()

In [ ]:
c["idex"].sample(10)

In [ ]:
clean_hplc()

In [ ]:
c["hplc"].switch_method(name="AH_cleaning")

In [ ]:
c["hplc"].switch_method(name="AH_default")

In [ ]:
c["hplc"].standby()

In [ ]:
c.move("pump_3", "waste_3", 2.5, speed=10)

In [7]:
import xdl
import chemputerxdl

In [8]:
from chemputerxdl.steps import Add, Transfer, CleanBackbone, CMove, CleanVessel, PrimePumpForAdd

In [ ]:
PrimePumpForAdd(reagent="", reagent_vessel="reactor", waste_vessel="waste_2", volume=0.1).execute(c)

In [ ]:
CleanVessel(vessel="dilution", solvent="acetonitrile", volume=5).execute(c)

In [9]:
CMove("dilution", "waste_3", 5).execute(c)

2020-09-04 12:54:14,283 ; pump_execution ; Moving 5.0 mL from dilution (None) to waste_3 (None). (Speed - Initial: 10.0, Mid: 40.0, End: 40.0)
INFO:chempiler:Moving 5.0 mL from dilution (None) to waste_3 (None). (Speed - Initial: 10.0, Mid: 40.0, End: 40.0)
DEBUG:main_logger.pv_logger:ChemputerValve valve_3 - Received command ('route', 3, -1), parameters = {'aspiration_wait': 2.0}).
DEBUG:main_logger.pv_logger:Clear errors DONE.
DEBUG:main_logger.pv_logger:ChemputerValve valve_3 - Switching to position 3.
DEBUG:main_logger.pv_logger:Actuation SUCCESS!
DEBUG:main_logger.pv_logger:Move DONE
DEBUG:main_logger.pv_logger:ChemputerPump pump_3 - Received command ('sink', 0), parameters = {'volume': 5.0, 'speed': 10.0, 'aspiration_wait': 2.0}).
DEBUG:main_logger.pv_logger:Clear errors DONE.
DEBUG:main_logger.pv_logger:Pump pump_3 is moving by 5.00mL at 10.0mL/min.
DEBUG:main_logger.pv_logger:Pump pump_3 is moving to 5.00mL at 10.0mL/min.
DEBUG:main_logger.pv_logger:Actuation SUCCESS!
DEBUG:mai

True

In [17]:
CMove("acetonitrile", "pump_3", volume=2, aspiration_speed=50).execute(c)

2020-09-04 13:06:43,265 ; pump_execution ; Moving 2.0 mL from acetonitrile (None) to pump_3 (None). (Speed - Initial: 50.0, Mid: 40.0, End: 40.0)
INFO:chempiler:Moving 2.0 mL from acetonitrile (None) to pump_3 (None). (Speed - Initial: 50.0, Mid: 40.0, End: 40.0)
DEBUG:main_logger.pv_logger:ChemputerValve valve_3 - Received command ('route', 5, -1), parameters = {'aspiration_wait': 2.0}).
DEBUG:main_logger.pv_logger:Clear errors DONE.
DEBUG:main_logger.pv_logger:ChemputerValve valve_3 - Switching to position 5.
DEBUG:main_logger.pv_logger:Actuation SUCCESS!
DEBUG:main_logger.pv_logger:Move DONE
DEBUG:main_logger.pv_logger:ChemputerPump pump_3 - Received command ('sink', 0), parameters = {'volume': 2.0, 'speed': 50.0, 'aspiration_wait': 2.0}).
DEBUG:main_logger.pv_logger:Clear errors DONE.
DEBUG:main_logger.pv_logger:Pump pump_3 is moving by 2.00mL at 50.0mL/min.
DEBUG:main_logger.pv_logger:Pump pump_3 is moving to 2.00mL at 50.0mL/min.
DEBUG:main_logger.pv_logger:Actuation SUCCESS!
DEB

True

In [19]:
CMove("pump_3", "waste_3", volume=2, dispense_speed=1, aspiration_speed=50).execute(c)

2020-09-04 13:07:17,970 ; pump_execution ; Moving 2.0 mL from pump_3 (None) to waste_3 (None). (Speed - Initial: 50.0, Mid: 40.0, End: 1.0)
INFO:chempiler:Moving 2.0 mL from pump_3 (None) to waste_3 (None). (Speed - Initial: 50.0, Mid: 40.0, End: 1.0)
DEBUG:main_logger.pv_logger:ChemputerValve valve_3 - Received command ('route', 0, -1), parameters = {'aspiration_wait': 2.0}).
DEBUG:main_logger.pv_logger:Clear errors DONE.
DEBUG:main_logger.pv_logger:ChemputerValve valve_3 - Switching to position 0.
DEBUG:main_logger.pv_logger:Actuation SUCCESS!
DEBUG:main_logger.pv_logger:Move DONE
DEBUG:main_logger.pv_logger:ChemputerPump pump_3 - Received command ('source', 0), parameters = {'volume': 2.0, 'speed': 1.0, 'aspiration_wait': 2.0}).
DEBUG:main_logger.pv_logger:Clear errors DONE.
DEBUG:main_logger.pv_logger:Pump pump_3 is moving by -2.00mL at 1.0mL/min.
DEBUG:main_logger.pv_logger:Pump pump_3 is moving to 0.00mL at 1.0mL/min.
DEBUG:main_logger.pv_logger:Actuation SUCCESS!
DEBUG:main_logg

True

In [10]:
CMove("dilution", "idex", 5).properties

{'from_vessel': 'dilution',
 'to_vessel': 'waste_3',
 'volume': 5.0,
 'move_speed': 40.0,
 'aspiration_speed': 10.0,
 'dispense_speed': 40.0,
 'from_port': None,
 'to_port': None,
 'through': '',
 'use_backbone': True,
 'repeats': 1}

In [ ]:
CleanBackbone(solvent="acetonitrile").human_readable()

In [6]:
CleanBackbone(solvent="acetonitrile").execute(c)

NameError: name 'CleanBackbone' is not defined

In [ ]:
Add(reagent="acetonitril", vessel="dilution", volume=0.1, priming_volume=1).execute(c)

In [ ]:
Transfer(from_vessel="reactor", to_vessel="dilution", volume=1).execute(c)

In [ ]:
Add(reagent="acetonitrile", vessel="dilution", volume=5).execute(c)

In [22]:
c.move('pump_3', 'waste_3', 4, speed=20)

2020-09-04 13:40:59,169 ; pump_execution ; Moving 4 mL from pump_3 () to waste_3 (). (Speed - Initial: 40, Mid: 40, End: 40)
INFO:chempiler:Moving 4 mL from pump_3 () to waste_3 (). (Speed - Initial: 40, Mid: 40, End: 40)
DEBUG:main_logger.pv_logger:ChemputerValve valve_3 - Received command ('route', 0, -1), parameters = {'aspiration_wait': 2.0}).
DEBUG:main_logger.pv_logger:Clear errors DONE.
DEBUG:main_logger.pv_logger:ChemputerValve valve_3 - Switching to position 0.
DEBUG:main_logger.pv_logger:Actuation SUCCESS!
DEBUG:main_logger.pv_logger:Move DONE
DEBUG:main_logger.pv_logger:ChemputerPump pump_3 - Received command ('source', 0), parameters = {'volume': 4, 'speed': 20, 'aspiration_wait': 2.0}).
DEBUG:main_logger.pv_logger:Clear errors DONE.
DEBUG:main_logger.pv_logger:Pump pump_3 is moving by -4.00mL at 20mL/min.
DEBUG:main_logger.pv_logger:Pump pump_3 is moving to 0.00mL at 20mL/min.
DEBUG:main_logger.pv_logger:Actuation SUCCESS!
DEBUG:main_logger.pv_logger:Move DONE.


[[(<ChemputerAPI.pump_valve_api.ChemputerValve at 0x23dad1cde50>,
   {'cmd': ('route', 0, -1), 'aspiration_wait': 2.0})],
 [(<ChemputerAPI.pump_valve_api.ChemputerPump at 0x23dad1cb880>,
   {'cmd': ('source', 0), 'volume': 4, 'speed': 20, 'aspiration_wait': 2.0})]]

In [25]:
c.move('pump_3', 'idex', 3, speed=10)

2020-09-04 13:43:32,962 ; pump_execution ; Moving 3 mL from pump_3 () to idex (). (Speed - Initial: 40, Mid: 40, End: 40)
INFO:chempiler:Moving 3 mL from pump_3 () to idex (). (Speed - Initial: 40, Mid: 40, End: 40)
DEBUG:main_logger.pv_logger:ChemputerValve valve_3 - Received command ('route', 4, -1), parameters = {'aspiration_wait': 2.0}).
DEBUG:main_logger.pv_logger:Clear errors DONE.
DEBUG:main_logger.pv_logger:ChemputerValve valve_3 - Switching to position 4.
DEBUG:main_logger.pv_logger:Actuation SUCCESS!
DEBUG:main_logger.pv_logger:Move DONE
DEBUG:main_logger.pv_logger:ChemputerPump pump_3 - Received command ('source', 0), parameters = {'volume': 3, 'speed': 10, 'aspiration_wait': 2.0}).
DEBUG:main_logger.pv_logger:Clear errors DONE.
DEBUG:main_logger.pv_logger:Pump pump_3 is moving by -3.00mL at 10mL/min.
DEBUG:main_logger.pv_logger:Pump pump_3 is moving to 0.00mL at 10mL/min.
DEBUG:main_logger.pv_logger:Actuation SUCCESS!
DEBUG:main_logger.pv_logger:Move DONE.


[[(<ChemputerAPI.pump_valve_api.ChemputerValve at 0x23dad1cde50>,
   {'cmd': ('route', 4, -1), 'aspiration_wait': 2.0})],
 [(<ChemputerAPI.pump_valve_api.ChemputerPump at 0x23dad1cb880>,
   {'cmd': ('source', 0), 'volume': 3, 'speed': 10, 'aspiration_wait': 2.0})]]

Exception in thread Exception in thread pump_2 TCP thread:
Traceback (most recent call last):
  File "c:\users\group\appdata\local\continuum\miniconda3\envs\hplc\lib\threading.py", line 932, in _bootstrap_inner
pump_3 TCP thread:
Traceback (most recent call last):
  File "c:\users\group\appdata\local\continuum\miniconda3\envs\hplc\lib\threading.py", line 932, in _bootstrap_inner
Exception in thread Exception in thread Exception in thread Exception in thread valve_3 TCP thread:
Traceback (most recent call last):
  File "c:\users\group\appdata\local\continuum\miniconda3\envs\hplc\lib\threading.py", line 932, in _bootstrap_inner
pump_1 TCP thread:
Traceback (most recent call last):
  File "c:\users\group\appdata\local\continuum\miniconda3\envs\hplc\lib\threading.py", line 932, in _bootstrap_inner
valve_1 TCP thread:
Traceback (most recent call last):
  File "c:\users\group\appdata\local\continuum\miniconda3\envs\hplc\lib\threading.py", line 932, in _bootstrap_inner
valve_2 TCP thread:
Tra

In [23]:
c.move('acetonitrile', 'pump_3', 5, speed=20)
c.move('pump_3', 'idex', 1, speed=1)

2020-09-04 13:41:28,500 ; pump_execution ; Moving 5 mL from acetonitrile () to pump_3 (). (Speed - Initial: 40, Mid: 40, End: 40)
INFO:chempiler:Moving 5 mL from acetonitrile () to pump_3 (). (Speed - Initial: 40, Mid: 40, End: 40)
DEBUG:main_logger.pv_logger:ChemputerValve valve_3 - Received command ('route', 5, -1), parameters = {'aspiration_wait': 2.0}).
DEBUG:main_logger.pv_logger:Clear errors DONE.
DEBUG:main_logger.pv_logger:ChemputerValve valve_3 - Switching to position 5.
DEBUG:main_logger.pv_logger:Actuation SUCCESS!
DEBUG:main_logger.pv_logger:Move DONE
DEBUG:main_logger.pv_logger:ChemputerPump pump_3 - Received command ('sink', 0), parameters = {'volume': 5, 'speed': 20, 'aspiration_wait': 2.0}).
DEBUG:main_logger.pv_logger:Clear errors DONE.
DEBUG:main_logger.pv_logger:Pump pump_3 is moving by 5.00mL at 20mL/min.
DEBUG:main_logger.pv_logger:Pump pump_3 is moving to 5.00mL at 20mL/min.
DEBUG:main_logger.pv_logger:Actuation SUCCESS!
DEBUG:main_logger.pv_logger:Move DONE
2020-

[[(<ChemputerAPI.pump_valve_api.ChemputerValve at 0x23dad1cde50>,
   {'cmd': ('route', 4, -1), 'aspiration_wait': 2.0})],
 [(<ChemputerAPI.pump_valve_api.ChemputerPump at 0x23dad1cb880>,
   {'cmd': ('source', 0), 'volume': 1, 'speed': 1, 'aspiration_wait': 2.0})]]

In [ ]:
c["hplc"].send("Exit")

In [ ]:
            CMove(
                from_vessel=self.dilution_vessel,
                to_vessel=self.injection_pump,
                volume=5, # dilution volume must be > 5
                aspiration_speed=10
            ),
            # move to idex (slowly)
            CMove(
                from_vessel=self.injection_pump,
                to_vessel=self.distribution_valve,
                volume=2.5,
                dispense_speed=0.5
            ),

In [ ]:
c.stirrer.set_stir_rate("dilution", 100)
c.stirrer.stir("dilution")

In [ ]:
c.stirrer.set_stir_rate("reactor", 200)
c.stirrer.stir("reactor")

In [ ]:
ChemputerAPI.flasks.ChemputerReactor.

In [1]:
from chemputeroptimizer.steps import FinalAnalysis

ModuleNotFoundError: No module named 'chemputeroptimizer.steps'

In [3]:
import chemputeroptimizer

In [ ]:
from chemputerxdl.utils.execution import get_nearest_node

In [ ]:
get_nearest_node(graph, self.dilution_vessel, "ChemputerIDEX")